In [1]:
using Test;

In [2]:
struct Point
    x::Int
    y::Int
end

GridData = Dict{Int,Int}
Grid = Dict{Point, GridData};

Intersections = Dict{Point, Int};

In [3]:
function parsewires(raw::Array{String,1})
    map(l -> split(l, ","), raw)
end

parsewires (generic function with 1 method)

In [4]:
function diranddist(instruction::AbstractString)::Tuple{AbstractString, Int}
    string(instruction[1]), parse(Int, instruction[2:end])
end

@test diranddist("R75") == ("R", 75);

In [5]:
function wiredata(grid::Grid, point::Point, wire::Int, step::Int)::GridData
    data = get(grid, point, GridData())
    
    data[wire] = step
    
    data
end

examplepoint = Point(0, 1)
examplegrid_empty = Grid()

wire, step = 1, 1

@test wiredata(examplegrid_empty, examplepoint, wire, step) == GridData(wire => step);

examplegrid = Grid()
examplegrid[examplepoint] = GridData(2 => 5)

@test wiredata(examplegrid, examplepoint, wire, step) == GridData(wire => step, 2 => 5);

In [6]:
function distance(a::Point, b::Point)::Int
    abs(a.x - b.x) + abs(a.y - b.y)
end

examplep1 = Point(1, 5)
examplep2 = Point(-6, 2)

@test distance(examplep1, examplep2) == 10;

In [7]:
function solve(wires)
    grid = Grid()
    
    for (wire, instructions) in enumerate(wires)
        x, y, step = 0, 0, 0

        for instruction in instructions
            dir, dist = diranddist(instruction)
            
            step = step + 1
            
            if dir == "R"
                for _ in 1:dist
                    x = x + 1

                    point = Point(x, y)
                    data = wiredata(grid, point, wire, step)

                    grid[point] = data
                end

            elseif dir == "L"
                for _ in 1:dist
                    x = x - 1

                    point = Point(x, y)
                    data = wiredata(grid, point, wire, step)          

                    grid[point] = data
                end

            elseif dir == "U"
                for _ in 1:dist
                    y = y + 1

                    point = Point(x, y)
                    data = wiredata(grid, point, wire, step)

                    grid[point] = data
                end

            elseif dir == "D"
                for _ in 1:dist
                    y = y - 1

                    point = Point(x, y)
                    data = wiredata(grid, point, wire, step)

                    grid[point] = data
                end
            else
                error("Got invalid direction: $dir\n")
            end
        end
    end
    
    grid
end

solve (generic function with 1 method)

In [8]:
function mindist(grid::Grid)
    intersections = filter(p -> length(values(p.second)) > 1, grid)
    
    origin = Point(0, 0)
    distances = map(p -> distance(origin, p), collect(keys(intersections)))
        
    minimum(distances)
end

example1 = parsewires([
    "R75,D30,R83,U83,L12,D49,R71,U7,L72",
    "U62,R66,U55,R34,D71,R55,D58,R83"
])

example2 = parsewires([
    "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51",
    "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"
])

grid1 = solve(example1)
@test mindist(grid1) == 159;

grid2 = solve(example2)
@test mindist(grid2) == 135;

In [10]:
# Part 1
f = open("day_3_input.txt");

wires = readlines(f);
wires = parsewires(wires);

grid = solve(wires);
results = mindist(grid)

print("Part 1: $results")

Part 1: 258